In [22]:
from typing_extensions import TypedDict
from typing import List

from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model

from pydantic import BaseModel

llm = init_chat_model("openai:gpt-4o")

In [23]:
class State(TypedDict):
    dish: str
    ingredients: list[dict]
    recipe_steps: str
    plating_instructions: str


class Ingredient(BaseModel):
    name: str
    quantity: str
    unit: str


class IngredientsOutput(BaseModel):
    ingredients: List[Ingredient]

In [24]:
def list_ingredients(state: State):
    structured_llm = llm.with_structured_output(IngredientsOutput)
    ing_res = structured_llm.invoke(
        f"""
        {state["dish"]}를 만드는데 필요한 재료들을 5~8개 사이로 정리해서 알려줘.
        """
    )
    return {"ingredients": ing_res.ingredients}


def create_recipe(state: State):
    recipe_res = llm.invoke(
        f"""
        {state["dish"]}를 만들기위한 재료 목록이야 {state["ingredients"]}
        이재료들을 조리 순서에 맞춰서 레시피를 완성해줘.
        10단계 이상 넘지않고 각각 200단어 정도의 길이로 만들어줘.
        최종 요리가 어떤 모습일지 100단어 이내로 설명하는 부분을 넣어줘.
        """
    )
    return {"recipe_steps": recipe_res.content}


def discribe_plating(state: State):
    plating_res = llm.invoke(
        f"""
        {state["dish"]}를 만든 과정은 {state["recipe_steps"]} 이런데
        최종 접시에 플레이팅 하면 좋은 전략을 3가지 제안해줘.
        """
    )

    return {"plating_instructions": plating_res.content}



In [25]:
graph_builder = StateGraph(State)

(
    graph_builder
    .add_node("list_ingredients", list_ingredients)
    .add_node("create_recipe", create_recipe)
    .add_node("discribe_plating", discribe_plating)

    .add_edge(START, "list_ingredients")
    .add_edge("list_ingredients", "create_recipe")
    .add_edge("create_recipe", "discribe_plating")
    .add_edge("discribe_plating", END)
)

graph = graph_builder.compile()

graph.invoke(
    {
        "dish": "hummus"
    }
)

{'dish': 'hummus',
 'ingredients': [Ingredient(name='Chickpeas', quantity='1', unit='can (15 oz)'),
  Ingredient(name='Tahini', quantity='1/4', unit='cup'),
  Ingredient(name='Lemon Juice', quantity='2', unit='tablespoons'),
  Ingredient(name='Garlic', quantity='1', unit='clove, minced'),
  Ingredient(name='Olive Oil', quantity='2', unit='tablespoons'),
  Ingredient(name='Salt', quantity='1/2', unit='teaspoon'),
  Ingredient(name='Ground Cumin', quantity='1/2', unit='teaspoon'),
  Ingredient(name='Water', quantity='2-3', unit='tablespoons')],
 'recipe_steps': '훔무스를 만드는 과정은 간단하지만 정성을 들이면 더욱 풍부한 맛을 낼 수 있습니다. 아래의 단계를 따라가며 부드럽고 크리미한 훔무스를 완성해보세요.\n\n1. 병아리콩 준비: 캔에 들어 있는 병아리콩을 체에 걸러 헹궈줍니다. 흐르는 물에 충분히 씻어낸 후 물기가 없도록 잘 털어줍니다. 이는 훔무스의 질감을 더욱 부드럽게 만들어 줍니다.\n\n2. 레몬 주스로 기본 맛 만들기: 레몬 주스를 작은 볼에 짜 넣고, 다진 마늘 1쪽 분량을 함께 섞어 마늘 레몬 소스를 만듭니다. 이 혼합물이 원래 병아리콩의 맛을 부드럽게 만들어주고 감칠맛을 더해줍니다.\n\n3. 모든 재료 결합: 푸드 프로세서에 병아리콩, 타히니, 레몬 주스-마늘 소스를 함께 넣고, 올리브 오일, 소금, 커민을 추가합니다. 모든 재료가 잘 섞이도록 합니다.\n\n4. 혼합 시작: 푸드 프로세서를 작동시켜 